In [10]:
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.geometry import Geometry
import pandas as pd
from pandas.io.parsers import read_table

In [23]:
csv_path = './access_raw20180612.csv'
geometry_column = 'SHAPE'
spatial_reference = {'wkid': 4326}  # WGS84

In [45]:
# read data in using read_csv parser
df = pd.io.parsers.read_table(csv_path, header=0, sep=',', index_col=0)

# convert geomery from string to dictionary
df[geometry_column] = df[geometry_column].apply(lambda geom: Geometry(eval(geom)))
    
# set the geometry
df.spatial.set_geometry(geometry_column)

In [49]:
new_df = pd.DataFrame(df)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9820 entries, 0 to 9819
Data columns (total 20 columns):
OBJECTID              9820 non-null int64
collection_method     0 non-null float64
created_date          0 non-null float64
created_user          0 non-null float64
difficulty            9814 non-null object
difficulty_max        9814 non-null object
difficulty_min        6271 non-null object
difficulty_outlier    1654 non-null object
last_edited_date      0 non-null float64
last_edited_user      0 non-null float64
name                  513 non-null object
name_common           1811 non-null object
name_river            9814 non-null object
name_section          9812 non-null object
nav_link_aw           9820 non-null object
notes                 7 non-null object
reach_id              9820 non-null int64
side_of_river         0 non-null float64
type                  9820 non-null object
SHAPE                 9820 non-null geometry
dtypes: float64(6), geometry(1), int64(2), object

In [ ]:
class ReachAccessesSDF(pd.DataFrame):

    @classmethod
    def from_csv(cls, path, header=0, sep=',', index_col=0,
                 parse_dates=True, encoding=None, tupleize_cols=None,
                 infer_datetime_format=False, geometry_column='SHAPE'):
        """
        Read a CSV file and create a ReachAccessesSDF object instance. This table must
            contain a field named "reach_id" and another containing the geometry for
            the accesses.
        :param path: string file path or file handle / StringIO
        :param header: int, default 0
            Row to use as header (skip prior rows)
        :param sep: string, default ','
            Field delimiter
        :param index_col: int or sequence, default 0
            Column to use for index. If a sequence is given, a MultiIndex
            is used. Different default from read_table
        :param parse_dates: boolean, default True
            Parse dates. Different default from read_table
        :param encoding:
        :param tupleize_cols: boolean, default False
            write multi_index columns as a list of tuples (if True)
            or new (expanded format) if False)
        :param infer_datetime_format: boolean, default False
            If True and `parse_dates` is True for a column, try to infer the
            datetime format based on the first datetime string. If the format
            can be inferred, there often will be a large parsing speed-up.
        :param geometry_column: string, default 'SHAPE'
            Column containing the geometry.
        :return: ReachesSDF
        """
        from pandas.io.parsers import read_table

        df = read_table(path, header=header, sep=sep, parse_dates=parse_dates, index_col=index_col, encoding=encoding,
                        tupleize_cols=tupleize_cols, infer_datetime_format=infer_datetime_format)

        reach_id_field = 'reach_id'

        # check for necessary fields
        if reach_id_field not in df.columns:
            raise Exception('input table must contain the {} field'.format(reach_id_field))
        if geometry_column not in df.columns:
            raise Exception('input table does not contain a geometry column named {}'.format(geometry_column))

        # ensure the reach_id field is a string
        if df[reach_id_field].dtype != object:
            df[reach_id_field] = df[reach_id_field].astype(str)
            
        # create 

        # return a copy of this class with the data populated
        return cls(
            data=df[[col for col in df.columns if col != geometry_column]],
            geometry=df[geometry_column].apply(lambda value: Geometry(eval(value)))
        )